# Assignment 2 - DS4Biz Y63
## TextScraping_Classification
***

### Team Detail
**Team Name**: sompinandsomshine   
***
### *Student 1*
**Student ID**: 61070278   
**Student Full Name**: นายกิตติภณ สุรุ่งเรืองสกุล
***
### *Student 2*
**Student ID**: 61070330   
**Student Full Name**: นางสาวอิงฟ้า ภูติวรนาถ
***

#### link: https://quotes.toscrape.com/

# Data Collection

In [1]:
import requests
#https://requests.readthedocs.io/en/master/
import bs4
#https://www.crummy.com/software/BeautifulSoup/bs4/doc/

import pandas as pd

In [2]:
response = requests.get('https://quotes.toscrape.com/page/1/')
print(response)
print(type(response))
html_page = bs4.BeautifulSoup(response.content, 'html.parser')
# print(html_page)

<Response [200]>
<class 'requests.models.Response'>


In [3]:
list_page = []
cur = 1
while 1:
    url = 'https://quotes.toscrape.com/page/'+str(cur)+'/'
    response = requests.get(url)
    html_page = bs4.BeautifulSoup(response.content, 'html.parser')
    selector = 'body > div > div:nth-child(2) > div.col-md-8'
    tag = html_page.select_one(selector)
    if 'No quotes found!' in tag.text:
        break
    else:
        list_page.append(url)
        cur += 1


In [4]:
list_page

['https://quotes.toscrape.com/page/1/',
 'https://quotes.toscrape.com/page/2/',
 'https://quotes.toscrape.com/page/3/',
 'https://quotes.toscrape.com/page/4/',
 'https://quotes.toscrape.com/page/5/',
 'https://quotes.toscrape.com/page/6/',
 'https://quotes.toscrape.com/page/7/',
 'https://quotes.toscrape.com/page/8/',
 'https://quotes.toscrape.com/page/9/',
 'https://quotes.toscrape.com/page/10/']

In [5]:
df = pd.DataFrame(columns=['content','author','tags', 'link_author'])
df

,content,author,tags,link_author


In [6]:
# Content
def extract_content(html_page):
    selector = 'div.col-md-8 > div > span.text'
    tags = html_page.select(selector)
    content = []
    
    for tag in tags:
        content.append(tag.text.strip().replace('“', '').replace('”', ''))
        
    return content

In [7]:
# Author
def extract_author(html_page):
    selector = 'div.col-md-8 > div > span > small'
    tags = html_page.select(selector)
    author = []
    
    for tag in tags:
        author.append(tag.text.strip().replace('-', ' '))
        
    return author

In [8]:
# tags
def extract_tags(html_page):
    selector = 'div.col-md-8 > div > div > meta'
    tags = html_page.select(selector)
    quote_tags = []
    
    for tag in tags:
#         print(tag['content'])
        quote_tags.append(tag['content'])
        
    return quote_tags

In [9]:
# link_author
def extract_link_author(html_page):
    selector = 'div.col-md-8 > div > span > a'
    tags = html_page.select(selector)
    link_author = []
    
    for tag in tags:
#         print(tag['content'])
        link_author.append('https://quotes.toscrape.com/author/' + tag['href'].split('/')[-1] + '/')
        
    return link_author

In [10]:
# response = requests.get('https://quotes.toscrape.com/page/1/')
# html_page = bs4.BeautifulSoup(response.content, 'html.parser')

# extract_content(html_page)

In [11]:
def extract_quotes(url):
    response = requests.get(url)
    html_page = bs4.BeautifulSoup(response.content, 'html.parser')

    content = extract_content(html_page)
    author = extract_author(html_page)
    tags = extract_tags(html_page)
    link_author = extract_link_author(html_page)
    
    Alltags = {'content':content,'author':author,'tags':tags,'link_author':link_author}
    result = pd.DataFrame(Alltags)
    return result

In [12]:
for url in list_page:
    result = extract_quotes(url)
    df = df.append(result, ignore_index=True)
df.head()

,content,author,tags,link_author
0,The world as we have created it is a process o...,Albert Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling/
2,There are only two ways to live your life. One...,Albert Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen/
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Mon...


In [13]:
df.isna().sum()

content        0
author         0
tags           0
link_author    0
dtype: int64

## author

In [14]:
# df_author = pd.DataFrame(columns=['Born_date','Born_location','Author_description'])
# df_author

In [15]:
# df['Link_author'].iloc[0]

In [16]:
# author born date
def born_date(html_page):
    born_date = html_page.findAll("span", {"class": "author-born-date"})
    born_date = born_date[0].text.replace(',', '').replace(' ', '-')
    return born_date

In [17]:
# author born location
def born_location(html_page):
    born_location = html_page.findAll("span", {"class": "author-born-location"})
    born_location = born_location[0].text.replace(',', '|')
    return born_location

In [18]:
# author Description
def description(html_page):
    description = html_page.findAll("div", {"class": "author-description"})
    description = description[0].text.strip()
    return description

In [19]:
# author Description
def author_name(html_page):
    author_name = html_page.findAll("h3", {"class": "author-title"})
    author_name = author_name[0].text.strip().replace('-', ' ')
    return author_name

In [20]:
# def extract_author(url):
#     response = requests.get(url)
#     html_page = bs4.BeautifulSoup(response.content, 'html.parser')

#     date = born_date(html_page)
#     location = born_location(html_page)
#     description_text = description(html_page)
    
#     Alldetail = {'Born_date': date, 'Born_location': location, 'Author_description':description_text}
#     result = pd.DataFrame(Alldetail)
#     return result

In [21]:
list_born_date = []
list_born_location = []
list_description = []
list_author_name = []

for link_author in list(df['link_author'].unique()):
    
#     response = requests.get('https://quotes.toscrape.com/author/Albert-Einstein/')
    response = requests.get(link_author)
    html_page = bs4.BeautifulSoup(response.content, 'html.parser')
    
    list_born_date.append(born_date(html_page))
    list_born_location.append(born_location(html_page))
    list_description.append(description(html_page))
    list_author_name.append(author_name(html_page))

In [22]:
Alldetail = {'author': list_author_name, 'born_date': list_born_date, 'born_location': list_born_location, 'author_description':list_description}
author_df = pd.DataFrame(Alldetail)

In [23]:
author_df.head()

,author,born_date,born_location,author_description
0,Albert Einstein,March-14-1879,in Ulm| Germany,"In 1879, Albert Einstein was born in Ulm, Germ..."
1,J.K. Rowling,July-31-1965,in Yate| South Gloucestershire| England| The U...,See also: Robert GalbraithAlthough she writes ...
2,Jane Austen,December-16-1775,in Steventon Rectory| Hampshire| The United Ki...,Jane Austen was an English novelist whose work...
3,Marilyn Monroe,June-01-1926,in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...
4,André Gide,November-22-1869,in Paris| France,André Paul Guillaume Gide was a French author ...


In [24]:
df.head()

,content,author,tags,link_author
0,The world as we have created it is a process o...,Albert Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling/
2,There are only two ways to live your life. One...,Albert Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen/
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Mon...


In [25]:
# df.set_index('Author').join(other.set_index('key'))

In [26]:
# result['Author'].value_counts()

In [27]:
content_author_df = pd.merge(df, author_df, on='author', how='left')
content_author_df.head()

,content,author,tags,link_author,born_date,born_location,author_description
0,The world as we have created it is a process o...,Albert Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...,March-14-1879,in Ulm| Germany,"In 1879, Albert Einstein was born in Ulm, Germ..."
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling/,July-31-1965,in Yate| South Gloucestershire| England| The U...,See also: Robert GalbraithAlthough she writes ...
2,There are only two ways to live your life. One...,Albert Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...,March-14-1879,in Ulm| Germany,"In 1879, Albert Einstein was born in Ulm, Germ..."
3,"The person, be it gentleman or lady, who has n...",Jane Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen/,December-16-1775,in Steventon Rectory| Hampshire| The United Ki...,Jane Austen was an English novelist whose work...
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Mon...,June-01-1926,in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...


## save DataFarame to csv
เพื่อให้ง่ายต่อการใช้ในภายหลัง
- content.csv รายละเอียดของ quote
- author.csv รายละเอียดของผู้เขียน
- content_author.csv รายละเอียดของ quote และผู้เขียน

In [28]:
df.to_csv('content.csv', index=False)
author_df.to_csv('author.csv', index=False)
content_author_df.to_csv('content_author.csv', index=False)

In [2]:
df = pd.read_csv('content.csv')
df.head()

,content,author,tags,link_author
0,The world as we have created it is a process o...,Albert Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling/
2,There are only two ways to live your life. One...,Albert Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen/
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Mon...


In [30]:
df.isna().sum()

content        0
author         0
tags           3
link_author    0
dtype: int64

In [31]:
df[df['tags'].isna()]

,content,author,tags,link_author
27,It is impossible to live without failing at so...,J.K. Rowling,NaN,https://quotes.toscrape.com/author/J-K-Rowling/
42,You believe lies so you eventually learn to tr...,Marilyn Monroe,NaN,https://quotes.toscrape.com/author/Marilyn-Mon...
78,The question isn't who is going to let me; it'...,Ayn Rand,NaN,https://quotes.toscrape.com/author/Ayn-Rand/


### บันทึก tags
folder target

In [33]:
df['tags']

0                   change,deep-thoughts,thinking,world
1                                     abilities,choices
2              inspirational,life,live,miracle,miracles
3                         aliteracy,books,classic,humor
4                             be-yourself,inspirational
                            ...                        
95                                  better-life-empathy
96    books,children,difficult,grown-ups,write,write...
97                                                truth
98                                        inspirational
99                                           books,mind
Name: tags, Length: 100, dtype: object

In [34]:
df['tags'].to_csv(r'target/tags.txt', header=None, index=None, sep=' ', mode='a')

---

# Test

In [38]:
df['tags'].str.contains('love', na=False)

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: tags, Length: 100, dtype: bool

In [26]:
type(df['tags'])

pandas.core.series.Series

In [76]:
love_df = df.loc[df['tags'].str.contains('love', na=False)][['content', 'tags']]
love_df

,content,tags
6,It is better to be hated for what you are than...,"life,love"
10,"This life is what you make it. No matter what,...","friends,heartbreak,inspirational,life,love,sis..."
13,"You may not be her first, her last, or her onl...",love
16,"The opposite of love is not hate, it's indiffe...","activism,apathy,hate,indifference,inspirationa..."
17,"It is not a lack of love, but a lack of friend...","friendship,lack-of-friendship,lack-of-love,lov..."
20,"I love you without knowing how, or when, or fr...","love,poetry"
43,"If you can make a woman laugh, you can make he...","girls,love"
45,The real lover is the man who can thrill you b...,love
49,Love does not begin and end the way we seem to...,love
50,There is nothing I would not do for those who ...,"friendship,love"


---

In [88]:
import requests
import bs4

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

In [134]:
df

,content,author,tags,link_author
0,The world as we have created it is a process o...,Albert Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling/
2,There are only two ways to live your life. One...,Albert Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen/
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Mon...
...,...,...,...,...
95,You never really understand a person until you...,Harper Lee,better-life-empathy,https://quotes.toscrape.com/author/Harper-Lee/
96,You have to write the book that wants to be wr...,Madeleine L'Engle,"books,children,difficult,grown-ups,write,write...",https://quotes.toscrape.com/author/Madeleine-L...
97,Never tell the truth to people who are not wor...,Mark Twain,truth,https://quotes.toscrape.com/author/Mark-Twain/
98,"A person's a person, no matter how small.",Dr. Seuss,inspirational,https://quotes.toscrape.com/author/Dr-Seuss/


In [181]:
raw_documents = df['content'].values.tolist()
y = df['tags'].values.tolist()


In [182]:
X_train, X_test, y_train, y_test = train_test_split(raw_documents, y, test_size=0.01, random_state=42)

In [183]:
len(X_train)

99

In [184]:
type(X_train)

list

In [186]:
X_test

['I declare after all there is no enjoyment like reading! How much sooner one tires of any thing than of a book! -- When I have a house of my own, I shall be miserable if I have not an excellent library.']

In [187]:
# raw_documents = [raw_documents]

In [188]:
from sklearn.feature_extraction.text import TfidfVectorizer
# we can pass in the same preprocessing parameters
vectorizer = TfidfVectorizer(stop_words="english",min_df = 0)
X = vectorizer.fit_transform(X_train)
# display some sample weighted values
print(X[0])

  (0, 426)	0.2668977408162666
  (0, 260)	0.17426922218423355
  (0, 104)	0.2668977408162666
  (0, 265)	0.19882810724569872
  (0, 310)	0.7345996468195913
  (0, 431)	0.18517278578881985
  (0, 311)	0.2668977408162666
  (0, 354)	0.2668977408162666
  (0, 340)	0.2668977408162666


In [189]:
X_test = vectorizer.transform(X_test)

In [190]:
X.shape[0]

99

In [191]:
from sklearn.metrics.pairwise import cosine_similarity
# Measure the cosine similarity between the first document vector and all of the others
max_cos = 0
best_row = 0
for row in range(1,X.shape[0]):
    cos = cosine_similarity(X_test, X[row])
    # best so far?
    if cos > max_cos:
        max_cos = cos
        best_row = row

In [192]:
print("Most similar document was row %d: cosine similarity = %.3f" % ( best_row, max_cos ) )

Most similar document was row 90: cosine similarity = 0.251


In [193]:
print(raw_documents[best_row][0:300])

The truth." Dumbledore sighed. "It is a beautiful and terrible thing, and should therefore be treated with great caution.


In [194]:
df.iloc[best_row]

content        The truth." Dumbledore sighed. "It is a beauti...
author                                              J.K. Rowling
tags                                                       truth
link_author      https://quotes.toscrape.com/author/J-K-Rowling/
Name: 90, dtype: object

In [196]:
y_test

['books,library,reading']